In [1]:
#import packages
import h2o, os, os.path
import sparkling_water as sw
#import spark classes
from pyspark import SparkConf, SparkContext

In [2]:
#create h2ocontext
h2oContext = sw.H2OContext.init_h2o_context_locally()

JVM stdout: /tmp/tmpPn3J61/h2o_kuba_started_from_python.out
JVM stderr: /tmp/tmpoBjdbl/h2o_kuba_started_from_python.err
15/10/02 10:17:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
15/10/02 10:17:31 WARN Utils: Your hostname, kuba-HP-ProBook-6550b resolves to a loopback address: 127.0.1.1; using 192.168.0.21 instead (on interface wlan0)
15/10/02 10:17:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
15/10/02 10:17:54 WARN H2OContext: Increasing 'spark.locality.wait' to value 30000
15/10/02 10:17:54 INFO H2OContext: Starting H2O services: Sparkling Water configuration:
  workers      : -1
  cloudName    : sparkling-water-kuba
  flatfile     : true
  basePort     : 54321
  cloudTimeout : 60000
  h2oNodeLog   : INFO
  h2oClientLog : WARN
  nthreads     : -1
  drddMulFactor: 10
15/10/02 10:18:11 INFO H2OContext: Detected 0 before, and 1 spark executors after! Retrying again...
15/10/02

H2O cluster uptime:,34 seconds 912 milliseconds
H2O cluster version:,3.2.0.3
H2O cluster name:,sparkling-water-kuba
H2O cluster total nodes:,1
H2O cluster total memory:,491.5 MB
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster healthy:,True
H2O Connection ip:,192.168.0.21
H2O Connection port:,54325


In [3]:
# Stop the H2OContext
h2oContext.stop()

In [6]:
# Upload data to the H2O cluster and obtained its representation as H2OFrame
# The path for the upload file method is either absolute or relative to the directory where this notebook was started
data = h2o.upload_file(path=os.path.abspath("../../examples/smalldata/prostate.csv"))


Parse Progress: [##################################################] 100%
Uploaded py3b197ecb-92a1-44e0-adb0-3c83334e1176 into cluster with 380 rows and 9 cols


In [7]:
# Print info about h2o frame
data.describe()

Rows: 380 Cols: 9

Chunk compression summary:



chunk_type,chunk_name,count,count_percentage,size,size_percentage
CBS,Bits,1,11.111112,118 B,2.4210093
C1N,1-Byte Integers (w/o NAs),5,55.555557,2.2 KB,45.958145
C2,2-Byte Integers,1,11.111112,828 B,16.9881
C2S,2-Byte Fractions,2,22.222223,1.6 KB,34.632744



Frame distribution summary:



,size,number_of_rows,number_of_chunks_per_column,number_of_chunks
192.168.0.14:54321,4.8 KB,380.0,1.0,9.0
mean,4.8 KB,380.0,1.0,9.0
min,4.8 KB,380.0,1.0,9.0
max,4.8 KB,380.0,1.0,9.0
stddev,0 B,0.0,0.0,0.0
total,4.8 KB,380.0,1.0,9.0



Column-by-Column Summary:



,ID,CAPSULE,AGE,RACE,DPROS,DCAPS,PSA,VOL,GLEASON
type,int,int,int,int,int,int,real,real,int
mins,1.0,0.0,43.0,0.0,1.0,1.0,0.3,0.0,0.0
maxs,380.0,1.0,79.0,2.0,4.0,2.0,139.7,97.6,9.0
sigma,109.840793879,0.491074338963,6.52707126917,0.308773258025,1.00010761815,0.310656449351,19.9975726686,18.3476199673,1.09195337443
zero_count,0,227,0,3,0,0,0,167,2
missing_count,0,0,0,0,0,0,0,0,0


In [8]:
# Do some changes to the original frame
# Remove ID from training frame
train = data.drop("ID")

In [9]:
# Do some changes to the original frame
# For VOL & GLEASON, a zero really means "missing"
vol = train['VOL']
vol[vol == 0] = None
gle = train['GLEASON']
gle[gle == 0] = None

In [10]:
# Do some changes to the original frame
# Convert CAPSULE to a logical factor
train['CAPSULE'] = train['CAPSULE'].asfactor()

In [11]:
# See that new frame is changed
train.describe()

Rows: 380 Cols: 8

Chunk compression summary:



chunk_type,chunk_name,count,count_percentage,size,size_percentage
CBS,Bits,1,12.5,118 B,1.3762538
C1N,1-Byte Integers (w/o NAs),5,62.5,2.2 KB,26.125496
C8D,64-bit Reals,2,25.0,6.1 KB,72.49825



Frame distribution summary:



,size,number_of_rows,number_of_chunks_per_column,number_of_chunks
192.168.0.14:54321,8.4 KB,380.0,1.0,8.0
mean,8.4 KB,380.0,1.0,8.0
min,8.4 KB,380.0,1.0,8.0
max,8.4 KB,380.0,1.0,8.0
stddev,0 B,0.0,0.0,0.0
total,8.4 KB,380.0,1.0,8.0



Column-by-Column Summary:



,CAPSULE,AGE,RACE,DPROS,DCAPS,PSA,VOL,GLEASON
type,enum,int,int,int,int,real,real,int
mins,0.0,43.0,0.0,1.0,1.0,0.3,0.0,0.0
maxs,1.0,79.0,2.0,4.0,2.0,139.7,97.6,9.0
sigma,0.491074338963,6.52707126917,0.308773258025,1.00010761815,0.310656449351,19.9975726686,18.3476199673,1.09195337443
zero_count,227,0,3,0,0,0,167,2
missing_count,0,0,0,0,0,0,0,0


In [12]:
# Transform H2OFrame back to DataFrame
dataframe = sw.Utils.h2oframe_2_dataframe(train)

In [13]:
# Get the dataframe's schema
dataframe.get_schema()

StructType(List(StructField(CAPSULE,StringType,false),StructField(AGE,ByteType,false),StructField(RACE,ByteType,false),StructField(DPROS,ByteType,false),StructField(DCAPS,ByteType,false),StructField(PSA,DoubleType,false),StructField(VOL,DoubleType,false),StructField(GLEASON,ByteType,false)))

In [14]:
# Get only dataframe's columns
dataframe.get_fields()

[u'CAPSULE', u'AGE', u'RACE', u'DPROS', u'DCAPS', u'PSA', u'VOL', u'GLEASON']

In [15]:
# Get number of columns
dataframe.get_num_fields()

8

In [16]:
# Download the content of Spark's DataFrame to file named data
home = os.path.expanduser("~")
filename =home + os.path.sep + "downloaded_dataset"
dataframe.download(filename)